## PSO optimizer tutorial.
### Burgers equation.
$$\frac{\partial u}{\partial t}+ u\frac{\partial u}{\partial x}=\mu\frac{\partial^2 u}{\partial x^2} $$
$$\mu=0.02/\pi$$
$$x\in[-1,1]$$
$$t\in[0,1]$$

*Initial and boundary conditions*
$$u(x, t=0)=-sin(\pi*x)$$
$$u(x=-1, t)=0$$
$$u(x=1, t)=0$$

import libraries and Solver modules.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy
import time
import pandas as pd
from scipy.integrate import quad
import sys
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
sys.path.append('../')
sys.path.pop()
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('tutorials'), '..')))

from tedeous.input_preprocessing import Equation
from tedeous.solver import Solver, PSO, Plots, grid_format_prepare
from tedeous.solution import Solution
from tedeous.device import solver_device, check_device
from tedeous.models import mat_model

Building grid, boundary conditions, equation.

In [2]:
solver_device('cuda')

mode = 'autograd'

mu = 0.02 / np.pi

##grid creation
x = torch.linspace(-1, 1, 21)
t = torch.linspace(0, 1, 21)

grid = grid_format_prepare([x, t], mode=mode).float()

##initial cond
bnd1 = torch.cartesian_prod(x, torch.tensor([0.])).float()
bndval1 = -torch.sin(np.pi * bnd1[:, 0])

##boundary cond
bnd2 = torch.cartesian_prod(torch.tensor([-1.]), t).float()
bndval2 = torch.zeros_like(bnd2[:, 0])

##boundary cond
bnd3 = torch.cartesian_prod(torch.tensor([1.]), t).float()
bndval3 = torch.zeros_like(bnd3[:, 0])

## collecting all conditions in:
# bnd (boundary points),
# bop (boundary opertor if exists),
# bndval (boundary value),
# var (variable number in system case),
# cond-n type ('dirichlet', 'operator', 'periodic', 'data')

bconds = [[bnd1, bndval1, 'dirichlet'],
            [bnd2, bndval2, 'dirichlet'],
            [bnd3, bndval3, 'dirichlet']]

## equation part
burgers_eq = {
    'du/dt**1':
        {
            'coeff': 1.,
            'du/dt': [1],
            'pow': 1,
            'var': 0
        },
    '+u*du/dx':
        {
            'coeff': 1,
            'u*du/dx': [[None], [0]],
            'pow': [1, 1],
            'var': [0, 0]
        },
    '-mu*d2u/dx2':
        {
            'coeff': -mu,
            'd2u/dx2': [0, 0],
            'pow': 1,
            'var': 0
        }
}


CUDA is available and used.


exact solution

In [3]:
def exact(grid):
    mu = 0.02 / np.pi

    def f(y):
        return np.exp(-np.cos(np.pi * y) / (2 * np.pi * mu))

    def integrand1(m, x, t):
        return np.sin(np.pi * (x - m)) * f(x - m) * np.exp(-m ** 2 / (4 * mu * t))

    def integrand2(m, x, t):
        return f(x - m) * np.exp(-m ** 2 / (4 * mu * t))

    def u(x, t):
        if t == 0:
            return -np.sin(np.pi * x)
        else:
            return -quad(integrand1, -np.inf, np.inf, args=(x, t))[0] / quad(integrand2, -np.inf, np.inf, args=(x, t))[
                0]

    solution = []
    for point in grid:
        solution.append(u(point[0].item(), point[1].item()))

    return torch.tensor(solution)

When grid, equation, boundary conditions  exist, we should call preprocessing class Equation with method set_strategy and initialize model.

In [4]:
equation = Equation(grid, burgers_eq, bconds).set_strategy(mode)

## model part
if mode in ('NN', 'autograd'):
    model = torch.nn.Sequential(
        torch.nn.Linear(2, 10),
        torch.nn.Tanh(),
        torch.nn.Linear(10, 10),
        torch.nn.Tanh(),
        torch.nn.Linear(10, 10),
        torch.nn.Tanh(),
        torch.nn.Linear(10, 1)
    )
else:
    model = mat_model(grid, equation)

After that, we can initialize optimizer, it may be one of torch optimizers or custom form *tedeous.optimizers* module.
If you want to run optimizatoin process with default settings,
you will be able to set it as string ("Adam", "SGD", "LBFGS", "PSO")

*Here is main aspects of particle swarm optimizer realization*
1. For optimization, the swarm *pop_size* is built based on initial model weights with adding some *variance* (influence on search space)
2. Each individual in this swarm represents a candidate solution. At each iteration, the particles in the swarm exchange
information and use it to update their positions.
3.  Particle $\theta^t$ at iteration $t$ is changed by three factors: its own velocity inertia $\beta \upsilon^t$
, its best-known position $p_{best}$ in the search-space, as well as the
entire swarm’s best-known position $g_{best}$:
$$\upsilon^{t+1} = \beta*\upsilon^{t} + (1-\beta)*(c1*r1(p_{best} − \theta^t) + c2*r2(g_{best} − \theta^t))$$
where *c1* and *c2* are the cognitive and social coefficients, respectively, referred to jointly as the behavioral
coefficients, and *r1* and *r2* are uniformly distributed random numbers in range (-*variance*, *variance*). Then the particle position is updated as:
$$\theta^{t+1} = \theta^t + \upsilon^{t+1}$$
4. PSO can be combined with gradient descent to train neural networks:
$$v^{t+1} = \beta*\upsilon^{t} + (1-\beta)*(c1*r1(p_{best} − \theta^t) + c2*r2(g_{best} − \theta^t)) − \alpha*\nabla Loss(\theta^t)$$
where $\alpha$ is *lr*.

Based on formulaes above, here is matching formulaes coef-nts with *PSO* algorithm parameters:
1. pop_size (int, optional): The swarm. Defaults to 30.
2. b (float, optional): Inertia of the particles. Defaults to 0.9.
3. c1 (float, optional): The *p-best* coeficient. Defaults to 0.08.
4. c2 (float, optional): The *g-best* coeficient. Defaults to 0.5.
5. c_decrease (bool, optional): Flag for update_pso_params method. Defautls to False.
6. variance (float, optional): Variance parameter for swarm creation
based on init model, ifluence on r1 and r2 coeff-nts. Defaults to 1.
7. lr (float, optional): Learning rate for gradient descent. Defaults to 1e-3.
    If 0, there will be only PSO optimization without gradients.
8. epsilon (float, optional): some add to gradient descent like in Adam optimizer. Defaults to 1e-8.

After preliminaries, to sart solving the equation, we should call Solver class with method solve:

In [5]:
img_dir=os.path.join(os.path.dirname('tutorials'), 'Burg_eq_img')

if not(os.path.isdir(img_dir)):
    os.mkdir(img_dir)

model = Solver(grid, equation, model, mode).solve(
    lambda_bound=10,
    verbose=True,
    learning_rate=1e-3,
    use_cache=False,
    print_every=1000,
    tmax=10000,
    patience=5,
    optimizer_mode='Adam',
    image_save_dir=img_dir)

u_exact = exact(grid).to('cuda')

u_exact = check_device(u_exact).reshape(-1)

u_pred = check_device(model(grid)).reshape(-1)

error_rmse = torch.sqrt(torch.sum((u_exact - u_pred)**2)) / torch.sqrt(torch.sum(u_exact**2))

print('RMSE_grad= ', error_rmse.item())

[2023-11-23 18:27:25.861406] initial (min) loss is 1.9947898387908936
[2023-11-23 18:27:25.873439] Print every 1000 step
Step = 0 loss = 1.994790 normalized loss line= -0.000000x+1.000000. There was 0 stop dings already.
[2023-11-23 18:27:35.200447] Print every 1000 step
Step = 1000 loss = 0.240816 normalized loss line= -0.000271x+1.026659. There was 0 stop dings already.
[2023-11-23 18:27:44.604552] Print every 1000 step
Step = 2000 loss = 0.173032 normalized loss line= -0.000313x+1.030409. There was 0 stop dings already.
[2023-11-23 18:27:53.946474] Print every 1000 step
Step = 3000 loss = 0.036074 normalized loss line= -0.003048x+1.291535. There was 0 stop dings already.
[2023-11-23 18:28:03.309823] Print every 1000 step
Step = 4000 loss = 0.012178 normalized loss line= -0.000818x+1.092276. There was 0 stop dings already.
[2023-11-23 18:28:12.676755] Print every 1000 step
Step = 5000 loss = 0.007998 normalized loss line= -0.000253x+1.024907. There was 0 stop dings already.
[2023-11-

C:\Users\user\AppData\Local\Temp\ipykernel_6388\2125334694.py:17: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return -quad(integrand1, -np.inf, np.inf, args=(x, t))[0] / quad(integrand2, -np.inf, np.inf, args=(x, t))[
C:\Users\user\AppData\Local\Temp\ipykernel_6388\2125334694.py:17: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return -quad(integrand1, -np.inf, np.inf, args=(x, t))[0] / quad(integrand2, -np.inf, np.inf, args=(x, t))[


RMSE_grad=  0.02456929041361502


for trained model we want to start PSO.

In [6]:
pso = PSO(
        pop_size=100,
        b=0.5,
        c2=0.05,
        variance=5e-3,
        c_decrease=True,
        lr=5e-3
    )
    
model = Solver(grid, equation, model, mode).solve(
    lambda_bound=10,
    verbose=True,
    eps=1e-6,
    use_cache=False,
    print_every=100,
    tmin=100,
    tmax=3000,
    patience=5,
    optimizer_mode=pso,
    image_save_dir=img_dir)

u_pred = check_device(model(grid)).reshape(-1)

error_rmse = torch.sqrt(torch.sum((u_exact - u_pred)**2)) / torch.sqrt(torch.sum(u_exact**2))

print('RMSE_pso= ', error_rmse.item())

Custom optimizer is activated
[2023-11-23 18:29:01.702648] initial (min) loss is 0.001300989417359233
[2023-11-23 18:29:02.520403] Print every 100 step
Step = 0 loss = 0.002916 normalized loss line= 0.000329x+0.435457. There was 0 stop dings already.
[2023-11-23 18:30:24.662127] Oscillation near the same loss
[2023-11-23 18:30:24.663128] Print every 100 step
Step = 100 loss = 0.002114 normalized loss line= -0.000000x+1.000000. There was 1 stop dings already.
[2023-11-23 18:31:45.337057] Print every 100 step
Step = 200 loss = 0.001896 normalized loss line= -0.001313x+1.142264. There was 1 stop dings already.
[2023-11-23 18:33:13.051870] Print every 100 step
Step = 300 loss = 0.001580 normalized loss line= -0.001884x+1.193018. There was 1 stop dings already.
[2023-11-23 18:34:41.517000] Print every 100 step
Step = 400 loss = 0.001341 normalized loss line= -0.001716x+1.150953. There was 1 stop dings already.
[2023-11-23 18:36:07.558068] Print every 100 step
Step = 500 loss = 0.001236 norm